# 🔬 Calculateur de doses - Études précliniques

Application simple pour calculer les besoins en produits pour vos études in vivo

In [1]:
import pandas as pd
from ipywidgets import (
    IntSlider, FloatSlider, Text, Button, VBox, HBox, 
    FloatText, Dropdown, Output, HTML
)
from IPython.display import display, Markdown, clear_output
from datetime import datetime

## ⚙️ Paramètres de l'étude

In [2]:
# Paramètres globaux
study_name = Text(
    value='Étude préclinique',
    placeholder='Nom de l\'étude',
    description='Nom:',
    style={'description_width': '150px'}
)

n_mice = IntSlider(
    value=8,
    min=1,
    max=50,
    step=1,
    description='Souris/groupe:',
    style={'description_width': '150px'}
)

weight = FloatSlider(
    value=20,
    min=1,
    max=100,
    step=1,
    description='Poids (g):',
    style={'description_width': '150px'}
)

duration = IntSlider(
    value=21,
    min=1,
    max=180,
    step=1,
    description='Durée (jours):',
    style={'description_width': '150px'}
)

margin = FloatSlider(
    value=10,
    min=0,
    max=50,
    step=5,
    description='Marge (%):',
    style={'description_width': '150px'}
)

display(
    VBox([
        study_name,
        n_mice,
        weight,
        duration,
        margin,
        HTML("<p style='color: #0066cc; font-size: 12px;'>💡 Conseil: Ajoutez 10-20% de marge pour compenser les pertes</p>")
    ])
)

## 📋 Définition des groupes

In [19]:
n_groups = IntSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description='Nombre de groupes:',
    style={'description_width': '150px'}
)

display(n_groups)

IntSlider(value=5, description='Nombre de groupes:', max=20, min=1, style=SliderStyle(description_width='150px…

In [20]:
# Initialisation globale
groups_list = []
groups_output = Output()

def create_group_widgets(n):
    groups_widgets = []
    
    for i in range(n):
        group_name = Text(
            value='Groupe ' + str(i+1),
            description='Nom:',
            style={'description_width': '80px'},
            layout={'width': '250px'}
        )
        
        dose = FloatText(
            value=0,
            min=0,
            max=1000,
            step=5,
            description='Dose (mg/kg):',
            style={'description_width': '120px'},
            layout={'width': '200px'}
        )
        
        dosing = Dropdown(
            options=[('QD (Quotidien)', 1), ('BID (2x/jour)', 0.5)],
            value=1,
            description='Dosing:',
            style={'description_width': '80px'}
        )
        
        group_box = VBox([
            HTML('<b style="color: #0066cc;">━━━ Groupe ' + str(i+1) + ' ━━━</b>'),
            group_name,
            HBox([dose, dosing]),
            HTML('<br>')
        ])
        
        groups_widgets.append({
            'name': group_name,
            'dose': dose,
            'dosing': dosing,
            'box': group_box
        })
    
    return groups_widgets

def update_groups(change=None):
    global groups_list
    n = n_groups.value
    groups_list = create_group_widgets(n)
    
    with groups_output:
        clear_output()
        display(VBox([g['box'] for g in groups_list]))

n_groups.observe(update_groups, names='value')
update_groups()
display(groups_output)

Output()

## 📊 Calcul et résultats

In [21]:
def calculate_doses():
    if not groups_list:
        display(HTML('<p style="color: red;">❌ Aucun groupe défini</p>'))
        return
    
    wt_kg = weight.value / 1000
    results = []
    
    for idx, group in enumerate(groups_list, 1):
        group_name = group['name'].value
        dose = group['dose'].value
        dosing_freq = group['dosing'].value
        
        n_doses = duration.value / dosing_freq
        total_dose = dose * wt_kg * n_mice.value * n_doses
        total_dose_margin = total_dose * (1 + margin.value/100)
        
        dosing_label = 'QD' if dosing_freq == 1 else 'BID'
        margin_int = int(margin.value)
        
        results.append({
            'Groupe': 'G' + str(idx) + ': ' + group_name,
            'Dose (mg/kg)': dose,
            'Dosing': dosing_label,
            'Composé (mg)': round(total_dose, 2),
            'Composé +' + str(margin_int) + '% (mg)': round(total_dose_margin, 2)
        })
    
    df = pd.DataFrame(results)
    return df

calc_button = Button(
    description='🧮 Calculer',
    button_style='success',
    tooltip='Lance le calcul des doses'
)

results_output = Output()

def on_calc_button_clicked(b):
    with results_output:
        clear_output()
        df = calculate_doses()
        
        if df is not None:
            study = study_name.value
            display(Markdown('### Résultats: **' + study + '**'))
            display(df)
            
            margin_int = int(margin.value)
            total_col = 'Composé +' + str(margin_int) + '% (mg)'
            total = df[total_col].sum()
            
            display(HTML('<h4>📦 Quantité totale à commander: <b style="color: #0066cc; font-size: 18px;">' + str(round(total, 1)) + ' mg</b></h4>'))
            
            export_csv_btn = Button(
                description='📥 CSV',
                button_style='info',
                tooltip='Exporte en CSV'
            )
            
            export_excel_btn = Button(
                description='📥 Excel',
                button_style='info',
                tooltip='Exporte en Excel'
            )
            
            def export_csv(b):
                study_clean = study_name.value.replace(' ', '_')
                date_str = datetime.now().strftime('%Y%m%d')
                filename = 'doses_' + study_clean + '_' + date_str + '.csv'
                df.to_csv(filename, index=False)
                print('✅ Fichier exporté: ' + filename)
            
            def export_excel(b):
                study_clean = study_name.value.replace(' ', '_')
                date_str = datetime.now().strftime('%Y%m%d')
                filename = 'doses_' + study_clean + '_' + date_str + '.xlsx'
                df.to_excel(filename, index=False, sheet_name='Doses')
                print('✅ Fichier exporté: ' + filename)
            
            export_csv_btn.on_click(export_csv)
            export_excel_btn.on_click(export_excel)
            
            display(HBox([export_csv_btn, export_excel_btn]))

calc_button.on_click(on_calc_button_clicked)
display(calc_button)
display(results_output)

Button(button_style='success', description='🧮 Calculer', style=ButtonStyle(), tooltip='Lance le calcul des dos…

Output()

## ℹ️ Informations

**QD (Once Daily)**: 1 dose par jour  
**BID (Twice Daily)**: 2 doses par jour  

**Formule de calcul**:  
Total = Dose (mg/kg) × Poids (kg) × Nombre d'animaux × Nombre de doses